In [38]:
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from typing import TypedDict
from dotenv import load_dotenv
load_dotenv()

True

In [39]:
# model
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.9)

In [40]:
# parser
parser = StrOutputParser()

In [41]:
# create a state

class BlogState(TypedDict):

    topic : str
    outline : str
    content : str
    score : int

In [42]:
def create_outline(state : BlogState) -> BlogState:

      # fetch title
      topic = state['topic']

      # prompt template
      template = PromptTemplate.from_template(
            template = "generate the detailed outline on the topic : {topic} for generating a blog"
      )

      # chain
      chain = template | model | parser

      # ask the llm for the outline
      outline = chain.invoke({"topic" : topic})

      # insert the outline
      state['outline'] = outline

      return state

In [43]:
def gen_blog(state : BlogState) -> BlogState:

      # fetch title
      topic = state['topic']
      outline = state['outline']

      # prompt template
      template = PromptTemplate.from_template(
            template = "generate an detailed blog on the topic : {topic} by using the following outline : {outline}"
      )

      # chain
      chain = template | model | parser

      # ask the llm for the blog content
      content = chain.invoke({"outline" : outline, "topic" : topic})

      # insert the content
      state['content'] = content

      return state

In [44]:
def get_scores(state : BlogState) -> BlogState:
      
      # fetch title
      content = state['content']
      outline = state['outline']

      # prompt template
      template = PromptTemplate.from_template(
            template = """
                  Rate the following blog post from 1 to 10 based on its adherence to the provided outline. 
                  Act as an experienced and discerning reader who expects high-quality, well-structured content.

                  Required Outline: {outline} Blog Content: {content}

                  Return your response in this format:

                  Rating: [Score/10]
                  NOTE : return only number
                        """
      )

      # chain
      chain = template | model | parser

      # ask the llm for the blog content
      score = chain.invoke({"outline" : outline, "content" : content})

      # insert the content
      state['score'] = score

      return state

In [45]:
# create our graph
graph = StateGraph(BlogState)

# add nodes
graph.add_node('create_outline', create_outline)
graph.add_node("gen_blog", gen_blog)
graph.add_node("get_scores", get_scores)

# add edges
graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'gen_blog')
graph.add_edge('gen_blog', "get_scores")
graph.add_edge('get_scores', END)

# compile
workflow = graph.compile()

In [46]:
# execute

intial_state = {"topic" : "How and ways to write prompt"}

final_state = workflow.invoke(intial_state)

In [47]:
final_state

{'topic': 'How and ways to write prompt',
 'outline': 'Here\'s a detailed outline on "How and Ways to Write Prompts for Generating a Blog Post," covering everything from foundational principles to advanced techniques and practical examples.\n\n---\n\n## Detailed Outline: How and Ways to Write Prompts for Generating a Blog Post\n\n### Introduction\n*   **A. The Rise of AI in Content Creation:** Briefly touch upon the increasing role of AI tools (like ChatGPT, Bard, Claude) in generating written content.\n*   **B. Importance of Effective Prompting:** Explain why a well-crafted prompt is crucial for high-quality, relevant, and useful AI-generated blog posts, contrasting it with vague prompts yielding generic outputs.\n*   **C. What This Outline Covers:** Overview of the process, techniques, and considerations for writing effective blog post prompts.\n\n### I. Pre-Prompt Planning: The Foundation of a Great Blog Post\n*   **A. Define the Blog Post\'s Core Purpose:**\n    *   **1. Topic/Subj

In [48]:
final_state['topic']

'How and ways to write prompt'

In [34]:
final_state['outline']

'## Detailed Outline: How and Ways to Write Prompts for Generating a Blog\n\n### Introduction\n*   **A. The Rise of AI in Content Creation:** Briefly touch upon the capabilities of AI in generating text.\n*   **B. The Importance of Effective Prompt Writing:**\n    *   Garnering high-quality, relevant, and unique output.\n    *   Saving time and iteration cycles.\n    *   Ensuring alignment with desired tone, style, and goals.\n*   **C. What This Guide Covers:** A comprehensive framework for crafting impactful blog prompts.\n\n### I. Understanding the Fundamentals of Effective Prompt Writing (General Principles)\n*   **A. Clarity & Precision:**\n    *   Avoid ambiguity: Use direct and unambiguous language.\n    *   Define terms: If using niche jargon, briefly explain or instruct AI to understand context.\n*   **B. Specificity & Detail:**\n    *   The more detail, the better the output (within reason).\n    *   Think like a human editor: What information would they need?\n*   **C. Concis

In [35]:
final_state['content']

'## Master the AI Whisperer: Your Comprehensive Guide to Writing Effective Blog Prompts\n\nThe digital content landscape is rapidly evolving, and at its forefront is the incredible power of Artificial Intelligence. Tools like ChatGPT, Bard, and other large language models (LLMs) have transformed how we conceptualize, draft, and publish content, allowing creators to generate text at unprecedented speeds.\n\nBut here\'s the catch: AI is only as good as the instructions it receives. This is where the art and science of **effective prompt writing** comes into play. A well-crafted prompt is the key to unlocking AI\'s full potential, ensuring you garner high-quality, relevant, and unique output that truly aligns with your vision. It saves invaluable time, reduces endless iteration cycles, and guarantees your content hits the desired tone, style, and strategic goals.\n\nThis guide is your comprehensive framework for becoming an AI whisperer, equipping you with the knowledge and tools to craft

In [37]:
final_state['score']

'As a strict, genuine, and experienced evaluator/reader, I have meticulously reviewed your blog post, "Master the AI Whisperer: Your Comprehensive Guide to Writing Effective Blog Prompts," against the detailed outline you provided.\n\nHere\'s my assessment:\n\n**Overall Adherence to Outline:**\nThe blog post demonstrates **exceptional adherence** to your detailed outline. Every section, every major point, and almost every sub-point outlined is present, addressed, and elaborated upon within the blog post. This level of precision and completeness is truly impressive and speaks to a very methodical approach to content creation.\n\n**Content Quality & Depth:**\n\n*   **Introduction:** Perfectly sets the stage, covering the rise of AI, the importance of prompt writing (with all sub-points), and clearly stating the guide\'s purpose. It\'s engaging and immediately establishes the value proposition.\n*   **Section I (Fundamentals):** Each principle (Clarity & Precision, Specificity & Detail, C